# 1. New Lock Object
C# 13 introduces a new `System.Threading.Lock` type for mutual exclusion. Historically, an object type was used for locking purposes, but now there’s a dedicated Lock type, which is expected to become the future standard for most locking.

In [1]:
// Before
public class LockExample 
{
    private readonly object _lock = new();

    public void DoStuff() 
    {
        lock (_lock)
        {
           Console.WriteLine("We're inside old lock");
        }          
    }
}

new LockExample().DoStuff();

We're inside old lock


In [1]:
using System.Threading;

// .NET 9
public class LockExample 
{
    private readonly Lock _lock = new();

    public void DoStuff() 
    {
        lock (_lock)
        {
            Console.WriteLine("We're inside .NET 9 lock");
        }          
    }
}

new LockExample().DoStuff();

We're inside .NET 9 lock


# 2. Task.WhenEach
Imagine you have a list of tasks that finish at different intervals. You want to process each one as soon as it completes. `WaitAll()` won’t work here since it waits for all tasks to finish. We can use `Task.WaitAny()` as a workaround to pick off the next one as it completes. C# 13 introduces a more elegant and efficient approach to handle this scenario: `Task.WhenEach`. Check out the example below.

In [ ]:
var tasks = Enumerable.Range(1, 5)
   .Select(async i =>
   {
     await Task.Delay(new Random().Next(1000, 5000));
     return $"Task {i} done";
   })
   .ToList();

var tasks2 = Enumerable.Range(1, 5)
   .Select(async i =>
   {
     await Task.Delay(new Random().Next(1000, 5000));
     return $"Task {i} done";
   })
   .ToList();

Console.WriteLine("Before .NET 9");
while(tasks.Count > 0)
{
   var completedTask = await Task.WhenAny(tasks);
   tasks.Remove(completedTask);
   Console.WriteLine(await completedTask);
}

Console.WriteLine(".NET 9");
await foreach (var completedTask in Task.WhenEach(tasks2))
   Console.WriteLine(await completedTask);

Before .NET 9
Task 1 done
Task 5 done
Task 4 done
Task 3 done
Task 2 done
.NET 9
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 1 done
